In [1]:
import pandas as pd
from scipy import stats
import numpy as np
from pandas.api.types import CategoricalDtype
import re
from helpers import get_renamed_blocks

In [2]:
bna = 'Kraftwerksnummer der Bundesnetzagentur'
sse = 'MaStR-Nr. der Stromerzeugungseinheit'
idcols = [bna, sse]

In [3]:
plr = pd.read_excel("../data/Kraftwerksliste_2022.xlsx", sheet_name=1, skiprows=9)
prtr = pd.read_excel("../data/2022-12-15_PRTR-Deutschland_Freisetzungen - Excel.xlsx")

In [4]:
plr[idcols] = plr[idcols].apply(lambda x: x.str.strip())

In [5]:
#plr

In [6]:
plr.loc[plr[bna] == "BNA0705 "]

,MaStR-Nr. der Stromerzeugungseinheit,Kraftwerksnummer der Bundesnetzagentur,Anzeige-Name der Stromerzeugungseinheit,PLZ der Einheit,Ort der Einheit,Straße der Einheit,Hausnummer der Einheit,Bundesland der Einheit,Datum der erstmaligen Inbetriebnahme der Einheit (Datum/Jahr),Datum der endgültigen Stilllegung der Einheit (Datum/Jahr),...,Auswertung Energieträger,Förderberechtigt nach EEG\n(ja/nein),Wärmeauskopplung (KWK)\n(ja/nein),Erneuerbarer Energieträger\n(ja/nein),Bruttoleistung in MW,Nettonennleistung (elektrische Wirkleistung) in MW,Technologie der Stromerzeugung,Volleinspeisung oder Teileinspeisung?,Anschlussnetzbetreiber,Spannungsebene


In [7]:
#pl0

In [8]:
pl1 = plr.dropna(subset=[bna])

In [9]:
new = plr[plr[bna].isnull()]

In [10]:
new2 = new.dropna(subset=['Bundesland der Einheit'])

In [11]:
new3 = new2[~new2['MaStR-Nr. der Stromerzeugungseinheit'].isin(["SEE9-Dummy-nicht_EEG", "SEE9-Dummy-EEG"])]

In [13]:
pl1

,MaStR-Nr. der Stromerzeugungseinheit,Kraftwerksnummer der Bundesnetzagentur,Anzeige-Name der Stromerzeugungseinheit,PLZ der Einheit,Ort der Einheit,Straße der Einheit,Hausnummer der Einheit,Bundesland der Einheit,Datum der erstmaligen Inbetriebnahme der Einheit (Datum/Jahr),Datum der endgültigen Stilllegung der Einheit (Datum/Jahr),...,Auswertung Energieträger,Förderberechtigt nach EEG\n(ja/nein),Wärmeauskopplung (KWK)\n(ja/nein),Erneuerbarer Energieträger\n(ja/nein),Bruttoleistung in MW,Nettonennleistung (elektrische Wirkleistung) in MW,Technologie der Stromerzeugung,Volleinspeisung oder Teileinspeisung?,Anschlussnetzbetreiber,Spannungsebene
0,BNA0302,BNA0302,Frimmersdorf,41517.0,Frimmersdorf,NaN,NaN,Nordrhein-Westfalen,1957,2011,...,Braunkohle,Nein,Nein,Nein,NaN,129.000,NaN,NaN,Westnetz GmbH,Hochspannung (HS)
1,BNA0303,BNA0303,Frimmersdorf,41517.0,Frimmersdorf,NaN,NaN,Nordrhein-Westfalen,1957,2011,...,Braunkohle,Nein,Nein,Nein,NaN,130.000,NaN,NaN,Westnetz GmbH,Hochspannung (HS)
2,BNA0304,BNA0304,Frimmersdorf,41517.0,Frimmersdorf,NaN,NaN,Nordrhein-Westfalen,1960,2011,...,Braunkohle,Nein,Nein,Nein,NaN,124.000,NaN,NaN,Amprion GmbH,Höchstspannung (HöS)
3,BNA0414,BNA0414,Westfalen,59071.0,Hamm-Uentrop,NaN,NaN,Nordrhein-Westfalen,1963,2011,...,Steinkohle,Nein,Nein,Nein,NaN,152.000,NaN,NaN,Westnetz GmbH,Hochspannung (HS)
4,BNA0415,BNA0415,Westfalen,59071.0,Hamm-Uentrop,NaN,NaN,Nordrhein-Westfalen,1963,2011,...,Steinkohle,Nein,Nein,Nein,NaN,152.000,NaN,NaN,Westnetz GmbH,Hochspannung (HS)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,SEE974753539035,BNA0801,Turbine 6,75175.0,Pforzheim,Hohwiesenweg,15,Baden-Württemberg,1990-01-18 00:00:00,2022-05-18 00:00:00,...,Steinkohle,Nein,Ja,Nein,29.700,26.900,Kondensationsmaschine mit Entnahme,Teileinspeisung (einschließlich Eigenverbrauch),NaN,Hochspannung
173,SEE943101874623,BNA1264,GT21,98528.0,Suhl,Fröhliche-Mann-Straße,2,Thüringen,1995-12-02 00:00:00,2022-03-31 00:00:00,...,Erdgas,Nein,Ja,Nein,4.550,4.400,Gasturbinen mit Abhitzekessel,Volleinspeisung,NaN,NaN
174,SEE963023008050,BNA1264,GT22,98528.0,Suhl,Fröhliche-Mann-Straße,2,Thüringen,1995-12-02 00:00:00,2022-03-31 00:00:00,...,Erdgas,Nein,Ja,Nein,4.550,4.400,Gasturbinen mit Abhitzekessel,Volleinspeisung,NaN,NaN
175,SEE969748285110,BNA1264,GT26,98528.0,Suhl,Fröhliche-Mann-Straße,2,Thüringen,1998-09-30 00:00:00,2022-03-31 00:00:00,...,Erdgas,Nein,Ja,Nein,4.550,4.400,Gasturbinen mit Abhitzekessel,Volleinspeisung,NaN,NaN


In [14]:
#new3

In [15]:
#new3.sort_values(by=['Bruttoleistung in MW'], ascending=False)

In [30]:
bm = pd.read_csv("block_plant_mapper.csv", engine="python")

In [17]:
plm = bm.merge(pl1, how="right", left_on="blockid", right_on=bna)

In [18]:
plm

,plantid,blockid,MaStR-Nr. der Stromerzeugungseinheit,Kraftwerksnummer der Bundesnetzagentur,Anzeige-Name der Stromerzeugungseinheit,PLZ der Einheit,Ort der Einheit,Straße der Einheit,Hausnummer der Einheit,Bundesland der Einheit,...,Auswertung Energieträger,Förderberechtigt nach EEG\n(ja/nein),Wärmeauskopplung (KWK)\n(ja/nein),Erneuerbarer Energieträger\n(ja/nein),Bruttoleistung in MW,Nettonennleistung (elektrische Wirkleistung) in MW,Technologie der Stromerzeugung,Volleinspeisung oder Teileinspeisung?,Anschlussnetzbetreiber,Spannungsebene
0,06-05-100-0081105,BNA0302,BNA0302,BNA0302,Frimmersdorf,41517.0,Frimmersdorf,NaN,NaN,Nordrhein-Westfalen,...,Braunkohle,Nein,Nein,Nein,NaN,129.000,NaN,NaN,Westnetz GmbH,Hochspannung (HS)
1,06-05-100-0081105,BNA0303,BNA0303,BNA0303,Frimmersdorf,41517.0,Frimmersdorf,NaN,NaN,Nordrhein-Westfalen,...,Braunkohle,Nein,Nein,Nein,NaN,130.000,NaN,NaN,Westnetz GmbH,Hochspannung (HS)
2,06-05-100-0081105,BNA0304,BNA0304,BNA0304,Frimmersdorf,41517.0,Frimmersdorf,NaN,NaN,Nordrhein-Westfalen,...,Braunkohle,Nein,Nein,Nein,NaN,124.000,NaN,NaN,Amprion GmbH,Höchstspannung (HöS)
3,06-05-900-0079017,BNA0414,BNA0414,BNA0414,Westfalen,59071.0,Hamm-Uentrop,NaN,NaN,Nordrhein-Westfalen,...,Steinkohle,Nein,Nein,Nein,NaN,152.000,NaN,NaN,Westnetz GmbH,Hochspannung (HS)
4,06-05-900-0079017,BNA0415,BNA0415,BNA0415,Westfalen,59071.0,Hamm-Uentrop,NaN,NaN,Nordrhein-Westfalen,...,Steinkohle,Nein,Nein,Nein,NaN,152.000,NaN,NaN,Westnetz GmbH,Hochspannung (HS)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,06-08-3042693,BNA0801,SEE974753539035,BNA0801,Turbine 6,75175.0,Pforzheim,Hohwiesenweg,15,Baden-Württemberg,...,Steinkohle,Nein,Ja,Nein,29.700,26.900,Kondensationsmaschine mit Entnahme,Teileinspeisung (einschließlich Eigenverbrauch),NaN,Hochspannung
168,661-110,BNA1264,SEE943101874623,BNA1264,GT21,98528.0,Suhl,Fröhliche-Mann-Straße,2,Thüringen,...,Erdgas,Nein,Ja,Nein,4.550,4.400,Gasturbinen mit Abhitzekessel,Volleinspeisung,NaN,NaN
169,661-110,BNA1264,SEE963023008050,BNA1264,GT22,98528.0,Suhl,Fröhliche-Mann-Straße,2,Thüringen,...,Erdgas,Nein,Ja,Nein,4.550,4.400,Gasturbinen mit Abhitzekessel,Volleinspeisung,NaN,NaN
170,661-110,BNA1264,SEE969748285110,BNA1264,GT26,98528.0,Suhl,Fröhliche-Mann-Straße,2,Thüringen,...,Erdgas,Nein,Ja,Nein,4.550,4.400,Gasturbinen mit Abhitzekessel,Volleinspeisung,NaN,NaN


In [19]:
nans = plm[plm['plantid'].isna()]

In [20]:
pln = plm.dropna(subset=["blockid"])

In [21]:
plp = pln.sort_values(by=[bna], ascending=False)

In [22]:
nans

,plantid,blockid,MaStR-Nr. der Stromerzeugungseinheit,Kraftwerksnummer der Bundesnetzagentur,Anzeige-Name der Stromerzeugungseinheit,PLZ der Einheit,Ort der Einheit,Straße der Einheit,Hausnummer der Einheit,Bundesland der Einheit,...,Auswertung Energieträger,Förderberechtigt nach EEG\n(ja/nein),Wärmeauskopplung (KWK)\n(ja/nein),Erneuerbarer Energieträger\n(ja/nein),Bruttoleistung in MW,Nettonennleistung (elektrische Wirkleistung) in MW,Technologie der Stromerzeugung,Volleinspeisung oder Teileinspeisung?,Anschlussnetzbetreiber,Spannungsebene
23,NaN,NaN,BNA0639,BNA0639,HKW Mannheim,68169.0,Mannheim,Otto-Hahn-Straße,1,Baden-Württemberg,...,Abfall,Nein,Ja,Nein,NaN,5.560,NaN,NaN,24/7 Netze GmbH,Mittelspannung (MS)
33,NaN,NaN,BNA0638,BNA0638,HKW Mannheim,68169.0,Mannheim,Otto-Hahn-Straße,1,Baden-Württemberg,...,Abfall,Nein,nein,Nein,NaN,2.780,NaN,NaN,Netrion GmbH,Mittelspannung (MS)
61,NaN,NaN,BNA0192,BNA0192,NaN,4509.0,Delitzsch,NaN,NaN,Sachsen,...,Biomasse,Ja,Ja,Ja,NaN,20.000,NaN,NaN,Mitteldeutsche Netzgesellschaft Strom mbH,Hochspannung (HS)
64,NaN,NaN,BNA0289a,BNA0289a,HKW West,60627.0,Frankfurt am Main,Gutleutstraße,231,Hessen,...,Sonstige Energieträger\n(nicht erneuerbar),Nein,Nein,Nein,NaN,19.700,NaN,NaN,NRM Netzdienste Rhein-Main GmbH,30
103,NaN,NaN,SEE996231694999,BNA1256,Dampfturbine WSK Kuhheide 1,16303.0,Schwedt,Kuhheide,1,Brandenburg,...,Abfall,Nein,Ja,Nein,14.000,13.300,Kondensationsmaschine mit Entnahme,Teileinspeisung,NaN,Hochspannung
117,NaN,NaN,SEE979772206050,BNA0492,GMA Generator 1,49479.0,Ibbenbüren,Osnabrücker Str.,141,Nordrhein-Westfalen,...,Grubengas,Ja,Nein,Nein,3.000,3.000,Verbrennungsmotor,Teileinspeisung,NaN,Mittelspannung
118,NaN,NaN,SEE994549176695,BNA0492,EVA,49479.0,Ibbenbüren,Osnabrücker Str.,141,Nordrhein-Westfalen,...,Grubengas,Ja,Nein,Nein,30.000,30.000,Kondensationsmaschine mit Entnahme,Teileinspeisung,NaN,NaN
133,NaN,NaN,SEE918337255669,BNA0520,Turbine 6,76187.0,Karlsruhe,Mitscherlichstraße,NaN,Baden-Württemberg,...,Wärme,Nein,Ja,Nein,20.000,18.000,Kondensationsmaschine mit Entnahme,Teileinspeisung (einschließlich Eigenverbrauch),NaN,Hochspannung
136,NaN,NaN,SEE900073920406,BNA0693,T5,24534.0,Neumünster,Bismarckstraße,51,Schleswig-Holstein,...,Steinkohle,Nein,Ja,Nein,21.800,20.365,Kondensationsmaschine mit Entnahme,Teileinspeisung (einschließlich Eigenverbrauch),NaN,Mittelspannung
141,NaN,NaN,SEE997842921603,BNA1233,EVE Premnitz,14727.0,Premnitz,Dr.-Herbert-Rein-Straße,1,Brandenburg,...,Abfall,Nein,Ja,Nein,25.000,25.000,Kondensationsmaschine mit Entnahme,Teileinspeisung (einschließlich Eigenverbrauch),NaN,Mittelspannung


In [23]:
#plp

In [24]:
cols = [i for i in range(3, 25)] + [1]
plq = plp.drop(plp.columns[cols],axis=1)

In [25]:
plq_cleaned = plq.dropna(subset=['MaStR-Nr. der Stromerzeugungseinheit'])

In [31]:
plq_cleaned.to_csv("mapper_retired.csv", index=False)

In [27]:
plq_cleaned

,plantid,MaStR-Nr. der Stromerzeugungseinheit
120,06-08-1479296,SEE901344829220
128,06-02-BERZ003800,SEE972471937317
135,661-81,SEE946665101624
129,06-02-BERZ003800,SEE999848168525
125,06-05-300-0890760,SEE937236372871
...,...,...
88,06-11-01-1105607,BNA0076
104,06-11-01-1105607,BNA0075
26,06-26200010633,BNA0059a
87,03-07-07244141350,BNA0012d
